In [1]:
from datasets import load_dataset

In [ ]:
dirs = """
pmg_ga_co/RCPiarsachALL
nnc_ga_mu/CDNamedEntities01
nnc_ga_mu/CDAn_tOileanach_part3_newLTS
nnc_ga_mu/CDAn_tOileanach_part4_newLTS
nnc_ga_mu/CDNamedEntities02
nnc_ga_mu/CDAlphabet
nnc_ga_mu/CDAn_tOileanach_part2_newLTS
nnc_ga_mu/corpas_beag_newLTS
nnc_ga_mu/CDSaytime

cmg_ga_mu/CMGOther
anb_ga_ul/new_text02
anb_ga_ul/text01-004
anb_ga_ul/text01
anb_ga_ul/text01-001
anb_ga_ul/text02
anb_ga_ul/named-entities
nnc_en_ie
"""

In [27]:
egset = load_dataset("abairxml.py", data_dir=f"/home/jim/tmp/cmg_ga_mu_corpusbeag")

Using custom data configuration default-187ee4831f664a6d
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b)


In [3]:
egset

DatasetDict({
    train: Dataset({
        features: ['file_id', 'words', 'phonemes', 'audio', 'dialect', 'language', 'speaker_id', 'audio_set'],
        num_rows: 1453
    })
})

In [28]:
from pathlib import Path
def check_paths(batch):
    audio_path = Path(batch["audio"])
    return audio_path.is_file()

egset = egset.filter(check_paths)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-96d34247636a824d.arrow


In [29]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["audio"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["words"]
    return batch

egset = egset.map(speech_file_to_array_fn, remove_columns=egset['train'].column_names)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-18df1d089931c98b.arrow


In [7]:
egset

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 1453
    })
})

In [9]:
import librosa
import numpy as np

In [30]:
def resample(batch):
    batch["duration"] = librosa.get_duration(y=batch["speech"], sr=batch["sampling_rate"])

    if batch["sampling_rate"] != 16_000:
        batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch["sampling_rate"], 16_000)
        batch["sampling_rate"] = 16_000

    return batch
egset = egset.map(resample, num_proc=4)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-da006be1caebfb5b.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-c2641d558f05e8f1.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-8d515b7aebd171c2.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-7b97b92998e9a71e.arrow


In [31]:
egset = egset.sort('duration')

In [32]:
egset['train'][0]['duration']

0.5309375

In [33]:
egset['train'][-1]['duration']

6.406125